In [ ]:
from google.colab import drive

# Montar o Google Drive
drive.mount	('/content/drive')
drive_path = '/content/drive/MyDrive/FilesForAiDetection'

In [ ]:
import os
from google.colab import files

files_for_detection = None
image_file_extensions = ['.jpg', '.png', '.jpeg']
video_file_extensions = ['.mp4']

# Carregar a imagem
uploaded = files.upload()
if not uploaded:
    print("No images uploaded, using default")
    if not os.path.exists(drive_path):
        print("Path for images and videos not found!")
        exit(0)
    files_for_detection = [os.path.join(drive_path, drive_file) for drive_file in os.listdir(drive_path)]
    if not files_for_detection:
        print("No images or videos found in the path!")
        exit(0)
else:
    files_for_detection = list(uploaded.keys())

OPENCV

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import os
import time

# Carregar o classificador em cascata para detecção de faces
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


def detect_faces(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=6)
    return faces

# Parâmetros do vídeo para salvar o resultado
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec para o formato MP4
out = None

# A imagem será salva no diretório atual
for filename in files_for_detection:
    if any([filename.endswith(file_extension) for file_extension in image_file_extensions]):
        image = cv2.imread(filename)

        # Converter BGR para GRAY para analisar corretamente
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        start_time = time.time()
        faces = detect_faces(image)
        end_time = time.time()
        print(f"Image detecting time: {end_time - start_time} seconds")


        # Desenhar retângulos ao redor das faces detectadas
        for (x, y, w, h) in faces:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 4)

        cv2_imshow(image)
    elif any([filename.endswith(file_extension) for file_extension in video_file_extensions]):
        cap = cv2.VideoCapture(filename)
        start_time = time.time()

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            faces = detect_faces(frame)

            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            if out is None:
                height, width = frame.shape[:2]
                out = cv2.VideoWriter('output_pessoas_detectadas.mp4', fourcc, 30.0, (width, height))

            # Escreve o frame processado no vídeo de saída
            out.write(frame)

        end_time = time.time()
        print(f"Video detecting time: {end_time - start_time} seconds")

        cap.release()
        out.release()
        files.download('output_pessoas_detectadas.mp4')


DLIB

In [ ]:
import dlib
import cv2
from google.colab.patches import cv2_imshow

hog_detector = dlib.get_frontal_face_detector()
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec para o formato MP4
out = None


for filename in files_for_detection:
    if any([filename.endswith(file_extension) for file_extension in image_file_extensions]):
        image = cv2.imread(filename)

        # Converter BGR para GRAY para analisar corretamente
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        start_time = time.time()
        faces = hog_detector(image, 1)
        end_time = time.time()
        print(f"Image detecting time: {end_time - start_time} seconds")

        # Desenhar retângulos ao redor das faces detectadas
        for face in faces:
            x1 = face.left()
            y1 = face.top()
            x2 = face.right()
            y2 = face.bottom()
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 4)

        cv2_imshow(image)
    elif any([filename.endswith(file_extension) for file_extension in video_file_extensions]):
        cap = cv2.VideoCapture(filename)

        start_time = time.time()
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            faces = hog_detector(frame, 1)

            # Desenhar retângulos ao redor das faces detectadas
            for face in faces:
                x1 = face.left()
                y1 = face.top()
                x2 = face.right()
                y2 = face.bottom()
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)

            if out is None:
                height, width = frame.shape[:2]
                out = cv2.VideoWriter('output_pessoas_detectadas.mp4', fourcc, 30.0, (width, height))

            # Escreve o frame processado no vídeo de saída
            out.write(frame)

        end_time = time.time()
        print(f"Video detecting time: {end_time - start_time} seconds")

        cap.release()
        out.release()
        files.download('output_pessoas_detectadas.mp4')